Evaluate the metrics In and Out of Distribution for MiniBOIA & Co

In [ ]:
import torch
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
import sys

sys.path.append("..")
from utils.train import convert_to_categories, compute_coverage, compute_coverage_hard
from datasets.boia import BOIA
from datasets.miniboia import MINIBOIA
from datasets.minikandinsky import MiniKandinsky
from datasets.kandinsky import Kandinsky
from datasets.shortcutmnist import SHORTMNIST
from datasets.clipkandinsky import CLIPKandinsky
from datasets.clipshortcutmnist import CLIPSHORTMNIST
from datasets.clipboia import CLIPBOIA
from datasets.clipminiboia import CLIPMiniboia
from models.boiadpl import BoiaDPL
from models.miniboiadpl import MiniBoiaDPL
from models.boialtn import BOIALTN
from models.miniboialtn import MiniBoiaLTN
from models.boiann import BOIAnn
from models.miniboiann import MiniBOIAnn
from models.miniboiacbm import MiniBoiaCBM
from models.boiacbm import BoiaCBM
from models.mnistcbm import MnistCBM
from models.mnistdpl import MnistDPL
from models.mnistltn import MnistLTN
from models.mnistnn import MNISTnn
from models.minikanddpl import MiniKandDPL
from models.kanddpl import KandDPL
from models.kandcbm import KandCBM
from models.kandltn import KANDltn
from models.kandnn import KANDnn
from sklearn.metrics import confusion_matrix
from argparse import Namespace
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

Class containing all the metrics which we are evaluating

In [ ]:
class Metrics:
    def __init__(
        self,
        concept_accuracy,
        label_accuracy,
        concept_f1_macro,
        concept_f1_micro,
        concept_f1_weighted,
        label_f1_macro,
        label_f1_micro,
        label_f1_weighted,
        collapse,
        collapse_hard,
        avg_nll,
    ):
        self.concept_accuracy = concept_accuracy
        self.label_accuracy = label_accuracy
        self.concept_f1_macro = concept_f1_macro
        self.concept_f1_micro = concept_f1_micro
        self.concept_f1_weighted = concept_f1_weighted
        self.label_f1_macro = label_f1_macro
        self.label_f1_micro = label_f1_micro
        self.label_f1_weighted = label_f1_weighted
        self.collapse = collapse
        self.collapse_hard = collapse_hard
        self.avg_nll = avg_nll


class BOIAMetrics(Metrics):
    def __init__(
        self,
        concept_accuracy,
        label_accuracy,
        concept_f1_macro,
        concept_f1_micro,
        concept_f1_weighted,
        label_f1_macro,
        label_f1_micro,
        label_f1_weighted,
        collapse,
        collapse_hard,
        collapse_forward,
        collapse_stop,
        collapse_left,
        collapse_right,
        collapse_hard_forward,
        collapse_hard_stop,
        collapse_hard_left,
        collapse_hard_right,
        mean_collapse,
        mean_hard_collapse,
        avg_nll,
    ):
        super(BOIAMetrics, self).__init__(
            concept_accuracy,
            label_accuracy,
            concept_f1_macro,
            concept_f1_micro,
            concept_f1_weighted,
            label_f1_macro,
            label_f1_micro,
            label_f1_weighted,
            collapse,
            collapse_hard,
            avg_nll,
        )
        self.collapse_forward = collapse_forward
        self.collapse_stop = collapse_stop
        self.collapse_left = collapse_left
        self.collapse_right = collapse_right
        self.collapse_hard_forward = collapse_hard_forward
        self.collapse_hard_stop = collapse_hard_stop
        self.collapse_hard_left = collapse_hard_left
        self.collapse_hard_right = collapse_hard_right
        self.mean_collapse = mean_collapse
        self.mean_hard_collapse = mean_hard_collapse


class KandMetrics(Metrics):
    def __init__(
        self,
        concept_accuracy,
        label_accuracy,
        concept_f1_macro,
        concept_f1_micro,
        concept_f1_weighted,
        label_f1_macro,
        label_f1_micro,
        label_f1_weighted,
        collapse,
        collapse_hard,
        avg_nll,
        collapse_shapes,
        collapse_hard_shapes,
        collapse_color,
        collapse_hard_color,
        mean_collapse,
        mean_collapse_hard,
    ):
        super(KandMetrics, self).__init__(
            concept_accuracy,
            label_accuracy,
            concept_f1_macro,
            concept_f1_micro,
            concept_f1_weighted,
            label_f1_macro,
            label_f1_micro,
            label_f1_weighted,
            collapse,
            collapse_hard,
            avg_nll,
        )
        self.collapse_shapes = collapse_shapes
        self.collapse_hard_shapes = collapse_hard_shapes
        self.collapse_color = collapse_color
        self.collapse_hard_color = collapse_hard_color
        self.mean_collapse = mean_collapse
        self.mean_collapse_hard = mean_collapse_hard

Function used to compute the concept collapse

In [ ]:
def compute_concept_collapse(true_concepts, predicted_concepts, multilabel=False):
    if multilabel:
        true_concepts = convert_to_categories(true_concepts.astype(int))
        predicted_concepts = convert_to_categories(predicted_concepts.astype(int))

    return 1 - compute_coverage(confusion_matrix(true_concepts, predicted_concepts))


def compute_hard_concept_collapse(true_concepts, predicted_concepts, multilabel=False):
    if multilabel:
        true_concepts = convert_to_categories(true_concepts.astype(int))
        predicted_concepts = convert_to_categories(predicted_concepts.astype(int))

    return 1 - compute_coverage_hard(
        confusion_matrix(true_concepts, predicted_concepts)
    )

Function used to plot confusion matrix

In [ ]:
def plot_confusion_matrix(
    true_labels,
    predicted_labels,
    classes,
    normalize=False,
    title=None,
    is_boia=False,
    cmap=plt.cm.Oranges,
):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    cm = np.zeros((len(classes), len(classes)))
    for i in range(len(true_labels)):
        cm[true_labels[i], predicted_labels[i]] += 1

    if normalize:
        cm = cm.astype("float")
        row_sums = cm.sum(axis=1)[:, np.newaxis]
        cm = np.where(row_sums == 0, 0, cm / row_sums)

    plt.figure(figsize=(8, 6))
    sns.set(font_scale=1.8)
    red_yellow_palette = sns.color_palette("OrRd", as_cmap=True)
    sns.heatmap(
        cm,
        annot=False,
        fmt=".2f" if normalize else "d",
        cmap=red_yellow_palette,
        cbar=True,
        xticklabels=classes,
        yticklabels=classes,
    )
    if title:
        plt.savefig(title, format="pdf")
    plt.xticks(rotation=0)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

Function used to compute the metrics

In [ ]:
def compute_metrics(
    true_labels,
    predicted_labels,
    true_concepts,
    predicted_concepts,
    avg_nll,
    dataset_name,
    model_name,
    seed,
):

    # multilabel or not
    multilabel_concept = False
    multilabel_label = False

    if dataset_name in ["boia", "miniboia", "clipboia", "clipminiboia"]:
        multilabel_concept = True
        multilabel_label = True

    if dataset_name in ["kandinsky", "minikandinsky", "clipkandinsky"]:
        collapse_true_concepts_list = torch.tensor(true_concepts)
        collapse_true_concepts_list = torch.split(collapse_true_concepts_list, 3, dim=1)
        collapse_pred_concepts_list = torch.tensor(predicted_concepts)
        collapse_pred_concepts_list = torch.split(collapse_pred_concepts_list, 3, dim=1)

        collapse_true_concepts_1 = collapse_true_concepts_list[0].flatten()
        collapse_true_concepts_2 = collapse_true_concepts_list[1].flatten()
        collapse_true_concepts = torch.stack(
            (collapse_true_concepts_1, collapse_true_concepts_2), dim=1
        )
        # to int
        collapse_true_concepts = (
            collapse_true_concepts[:, 0] * 3 + collapse_true_concepts[:, 1]
        )
        collapse_true_concepts = collapse_true_concepts.detach().numpy()

        collapse_pred_concepts_1 = collapse_pred_concepts_list[0].flatten()
        collapse_pred_concepts_2 = collapse_pred_concepts_list[1].flatten()
        collapse_pred_concepts = torch.stack(
            (collapse_pred_concepts_1, collapse_pred_concepts_2), dim=1
        )
        # to int
        collapse_pred_concepts = (
            collapse_pred_concepts[:, 0] * 3 + collapse_pred_concepts[:, 1]
        )
        collapse_pred_concepts = collapse_pred_concepts.detach().numpy()

        # total collapse
        collapse = compute_concept_collapse(
            collapse_true_concepts, collapse_pred_concepts, multilabel_concept
        )

        collapse_hard = compute_hard_concept_collapse(
            collapse_true_concepts, collapse_pred_concepts, multilabel_concept
        )

    else:
        # total collapse
        collapse = compute_concept_collapse(
            true_concepts, predicted_concepts, multilabel_concept
        )

        collapse_hard = compute_hard_concept_collapse(
            true_concepts, predicted_concepts, multilabel_concept
        )

    if dataset_name in ["boia", "miniboia", "clipboia", "clipminiboia"]:
        # additional metrics for boia and miniboia
        collapse_forward, collapse_hard_forward = compute_concept_collapse(
            true_concepts[:, :3], predicted_concepts[:, :3], True
        ), compute_hard_concept_collapse(
            true_concepts[:, :3], predicted_concepts[:, :3], True
        )
        collapse_stop, collapse_hard_stop = compute_concept_collapse(
            true_concepts[:, 3:9], predicted_concepts[:, 3:9], True
        ), compute_hard_concept_collapse(
            true_concepts[:, 3:9], predicted_concepts[:, 3:9], True
        )
        collapse_left, collapse_hard_left = compute_concept_collapse(
            true_concepts[:, 9:15], predicted_concepts[:, 9:15], True
        ), compute_hard_concept_collapse(
            true_concepts[:, 9:15], predicted_concepts[:, 9:15], True
        )
        collapse_right, collapse_hard_right = compute_concept_collapse(
            true_concepts[:, 15:21], predicted_concepts[:, 15:21], True
        ), compute_hard_concept_collapse(
            true_concepts[:, 15:21], predicted_concepts[:, 15:21], True
        )

        mean_collapse, mean_hard_collapse = np.mean(
            [collapse_forward, collapse_stop, collapse_left, collapse_right]
        ), np.mean(
            [
                collapse_hard_forward,
                collapse_hard_stop,
                collapse_hard_left,
                collapse_hard_right,
            ]
        )

    elif dataset_name in ["minikandinsky", "kandinsky", "clipkandinsky"]:
        # additional metrics for boia and miniboia
        collapse_color, collapse_hard_color = compute_concept_collapse(
            true_concepts[:, 3:6].reshape(-1),
            predicted_concepts[:, 3:6].reshape(-1),
            False,
        ), compute_hard_concept_collapse(
            true_concepts[:, 3:6].reshape(-1),
            predicted_concepts[:, 3:6].reshape(-1),
            False,
        )
        collapse_shapes, collapse_hard_shapes = compute_concept_collapse(
            true_concepts[:, :3].reshape(-1),
            predicted_concepts[:, :3].reshape(-1),
            False,
        ), compute_hard_concept_collapse(
            true_concepts[:, :3].reshape(-1),
            predicted_concepts[:, :3].reshape(-1),
            False,
        )

        mean_collapse, mean_collapse_hard = np.mean(
            [collapse_color, collapse_shapes]
        ), np.mean([collapse_hard_color, collapse_hard_shapes])

    if multilabel_concept:
        concept_accuracy, concept_f1_macro, concept_f1_micro, concept_f1_weighted = (
            0,
            0,
            0,
            0,
        )

        for i in range(true_concepts.shape[1]):
            concept_accuracy += accuracy_score(true_concepts[i], predicted_concepts[i])
            concept_f1_macro += f1_score(
                true_concepts[i], predicted_concepts[i], average="macro"
            )
            concept_f1_micro += f1_score(
                true_concepts[i], predicted_concepts[i], average="micro"
            )
            concept_f1_weighted += f1_score(
                true_concepts[i], predicted_concepts[i], average="weighted"
            )

        concept_accuracy = concept_accuracy / true_concepts.shape[1]
        concept_f1_macro = concept_f1_macro / true_concepts.shape[1]
        concept_f1_micro = concept_f1_micro / true_concepts.shape[1]
        concept_f1_weighted = concept_f1_weighted / true_concepts.shape[1]

        label_accuracy, label_f1_macro, label_f1_micro, label_f1_weighted = 0, 0, 0, 0
    elif dataset_name in ["kandinsky", "minikandinsky", "clipkandinsky"]:
        concept_accuracy_color = accuracy_score(
            true_concepts[:, 3:6].reshape(-1), predicted_concepts[:, 3:6].reshape(-1)
        )
        concept_f1_macro_color = f1_score(
            true_concepts[:, 3:6].reshape(-1),
            predicted_concepts[:, 3:6].reshape(-1),
            average="macro",
        )
        concept_f1_micro_color = f1_score(
            true_concepts[:, 3:6].reshape(-1),
            predicted_concepts[:, 3:6].reshape(-1),
            average="micro",
        )
        concept_f1_weighted_color = f1_score(
            true_concepts[:, 3:6].reshape(-1),
            predicted_concepts[:, 3:6].reshape(-1),
            average="weighted",
        )

        concept_accuracy_shape = accuracy_score(
            true_concepts[:, :3].reshape(-1), predicted_concepts[:, :3].reshape(-1)
        )
        concept_f1_macro_shape = f1_score(
            true_concepts[:, :3].reshape(-1),
            predicted_concepts[:, :3].reshape(-1),
            average="macro",
        )
        concept_f1_micro_shape = f1_score(
            true_concepts[:, :3].reshape(-1),
            predicted_concepts[:, :3].reshape(-1),
            average="micro",
        )
        concept_f1_weighted_shape = f1_score(
            true_concepts[:, :3].reshape(-1),
            predicted_concepts[:, :3].reshape(-1),
            average="weighted",
        )

        concept_accuracy = np.mean([concept_accuracy_color, concept_accuracy_shape])
        concept_f1_macro = np.mean([concept_f1_macro_color, concept_f1_macro_shape])
        concept_f1_micro = np.mean([concept_f1_micro_color, concept_f1_micro_shape])
        concept_f1_weighted = np.mean(
            [concept_f1_weighted_color, concept_f1_weighted_shape]
        )

    else:
        concept_accuracy = accuracy_score(true_concepts, predicted_concepts)
        concept_f1_macro = f1_score(true_concepts, predicted_concepts, average="macro")
        concept_f1_micro = f1_score(true_concepts, predicted_concepts, average="micro")
        concept_f1_weighted = f1_score(
            true_concepts, predicted_concepts, average="weighted"
        )

    if multilabel_label:
        for i in range(true_labels.shape[1]):
            label_accuracy += accuracy_score(true_labels[i], predicted_labels[i])
            label_f1_macro += f1_score(
                true_labels[i], predicted_labels[i], average="macro"
            )
            label_f1_micro += f1_score(
                true_labels[i], predicted_labels[i], average="micro"
            )
            label_f1_weighted += f1_score(
                true_labels[i], predicted_labels[i], average="weighted"
            )

        label_accuracy = label_accuracy / true_labels.shape[1]
        label_f1_macro = label_f1_macro / true_labels.shape[1]
        label_f1_micro = label_f1_micro / true_labels.shape[1]
        label_f1_weighted = label_f1_weighted / true_labels.shape[1]
    else:
        label_accuracy = accuracy_score(true_labels, predicted_labels)
        label_f1_macro = f1_score(true_labels, predicted_labels, average="macro")
        label_f1_micro = f1_score(true_labels, predicted_labels, average="micro")
        label_f1_weighted = f1_score(true_labels, predicted_labels, average="weighted")

    if dataset_name in ["boia", "miniboia", "clipboia", "clipminiboia"]:
        metrics = BOIAMetrics(
            concept_accuracy=concept_accuracy,
            label_accuracy=label_accuracy,
            concept_f1_macro=concept_f1_macro,
            concept_f1_micro=concept_f1_micro,
            concept_f1_weighted=concept_f1_weighted,
            label_f1_macro=label_f1_macro,
            label_f1_micro=label_f1_micro,
            label_f1_weighted=label_f1_weighted,
            collapse=collapse,
            collapse_hard=collapse_hard,
            collapse_forward=collapse_forward,
            collapse_stop=collapse_stop,
            collapse_right=collapse_right,
            collapse_left=collapse_left,
            collapse_hard_forward=collapse_hard_forward,
            collapse_hard_stop=collapse_hard_stop,
            collapse_hard_right=collapse_hard_right,
            collapse_hard_left=collapse_hard_left,
            mean_collapse=mean_collapse,
            mean_hard_collapse=mean_hard_collapse,
            avg_nll=avg_nll,
        )
    elif dataset_name in ["minikandinsky", "kandinsky", "clipkandinsky"]:
        metrics = KandMetrics(
            concept_accuracy=concept_accuracy,
            label_accuracy=label_accuracy,
            concept_f1_macro=concept_f1_macro,
            concept_f1_micro=concept_f1_micro,
            concept_f1_weighted=concept_f1_weighted,
            label_f1_macro=label_f1_macro,
            label_f1_micro=label_f1_micro,
            label_f1_weighted=label_f1_weighted,
            collapse=collapse,
            collapse_hard=collapse_hard,
            avg_nll=avg_nll,
            collapse_shapes=collapse_shapes,
            collapse_color=collapse_color,
            collapse_hard_shapes=collapse_hard_shapes,
            mean_collapse_hard=mean_collapse_hard,
            mean_collapse=mean_collapse,
            collapse_hard_color=collapse_hard_color,
        )
    else:
        metrics = Metrics(
            concept_accuracy=concept_accuracy,
            label_accuracy=label_accuracy,
            concept_f1_macro=concept_f1_macro,
            concept_f1_micro=concept_f1_micro,
            concept_f1_weighted=concept_f1_weighted,
            label_f1_macro=label_f1_macro,
            label_f1_micro=label_f1_micro,
            label_f1_weighted=label_f1_weighted,
            collapse=collapse,
            collapse_hard=collapse_hard,
            avg_nll=avg_nll,
        )

    if dataset_name in ["shortmnist"]:
        plot_confusion_matrix(
            true_concepts,
            predicted_concepts,
            classes=[i for i in range(10)],
            normalize=True,
            title=f"{model_name}_{dataset_name}_{seed}.pdf",
            is_boia=True,
        )
    elif dataset_name in ["boia", "miniboia"]:

        plot_confusion_matrix(
            convert_to_categories(true_concepts[:, :3].astype(int)),
            convert_to_categories(predicted_concepts[:, :3].astype(int)),
            ["" for i in range(2**3)],
            True,
            f"{model_name}_{dataset_name}_{seed}_forward.pdf",
        )
        plot_confusion_matrix(
            convert_to_categories(true_concepts[:, 3:9].astype(int)),
            convert_to_categories(predicted_concepts[:, 3:9].astype(int)),
            ["" for i in range(2**6)],
            True,
            f"{model_name}_{dataset_name}_{seed}_stop.pdf",
        )
        plot_confusion_matrix(
            convert_to_categories(true_concepts[:, 9:15].astype(int)),
            convert_to_categories(predicted_concepts[:, 9:15].astype(int)),
            ["" for i in range(2**6)],
            True,
            f"{model_name}_{dataset_name}_{seed}_left.pdf",
        )
        plot_confusion_matrix(
            convert_to_categories(true_concepts[:, 15:21].astype(int)),
            convert_to_categories(predicted_concepts[:, 15:21].astype(int)),
            ["" for i in range(2**6)],
            True,
            f"{model_name}_{dataset_name}_{seed}_right.pdf",
        )
    elif dataset_name in ["kandinsky", "minikandinsky"]:
        plot_confusion_matrix(
            true_concepts,
            predicted_concepts,
            classes=[i for i in range(10)],
            normalize=True,
            title=f"{model_name}_{dataset_name}_{seed}.pdf",
        )
    return metrics

Load the right dataset and the right model

In [ ]:
def get_dataset(datasetname, args):
    if datasetname.lower() == "boia":
        return BOIA(args)
    if datasetname.lower() == "miniboia":
        return MINIBOIA(args)
    if datasetname.lower() == "minikandinsky":
        return MiniKandinsky(args)
    if datasetname.lower() == "kandinsky":
        return Kandinsky(args)
    if datasetname.lower() == "shortmnist":
        return SHORTMNIST(args)
    if datasetname.lower() == "clipkandinsky":
        return CLIPKandinsky(args)
    if datasetname.lower() == "clipshortmnist":
        return CLIPSHORTMNIST(args)
    if datasetname.lower() == "clipboia":
        return CLIPBOIA(args)
    if datasetname.lower() == "clipminiboia":
        return CLIPMiniboia(args)

    raise NotImplementedError(f"Dataset {datasetname} missing")

In [ ]:
def get_model(modelname, encoder, args):
    if modelname.lower() == "boiadpl":
        return BoiaDPL(encoder=encoder, args=args)
    if modelname.lower() == "miniboiadpl":
        return MiniBoiaDPL(encoder=encoder, args=args)
    if modelname.lower() == "boialtn":
        return BOIALTN(encoder=encoder, args=args)
    if modelname.lower() == "miniboialtn":
        return MiniBoiaLTN(encoder=encoder, args=args)
    if modelname.lower() == "boiann":
        return BOIAnn(encoder=encoder, args=args)
    if modelname.lower() == "miniboiann":
        return MiniBOIAnn(encoder=encoder, args=args)
    if modelname.lower() == "boiacbm":
        return BoiaCBM(encoder=encoder, args=args)
    if modelname.lower() == "miniboiacbm":
        return MiniBoiaCBM(encoder=encoder, args=args)
    if modelname.lower() == "minikanddpl":
        return MiniKandDPL(encoder=encoder, args=args)
    if modelname.lower() == "kandltn":
        return KANDltn(encoder=encoder, args=args)
    if modelname.lower() == "kandnn":
        return KANDnn(encoder=encoder, args=args)
    if modelname.lower() == "kanddpl":
        return KandDPL(encoder=encoder, args=args)
    if modelname.lower() == "kandcbm":
        return KandCBM(encoder=encoder, args=args)
    if modelname.lower() == "mnistdpl":
        return MnistDPL(encoder=encoder, args=args)
    if modelname.lower() == "mnistltn":
        return MnistLTN(encoder=encoder, args=args)
    if modelname.lower() == "mnistnn":
        return MNISTnn(encoder=encoder, args=args)
    if modelname.lower() == "mnistcbm":
        return MnistCBM(encoder=encoder, args=args)

    raise NotImplementedError(f"Model {modelname} missing")

In [ ]:
args = Namespace(
    backbone="conceptizer",  #
    preprocess=0,
    finetuning=0,
    batch_size=256,
    n_epochs=20,
    validate=1,
    dataset="miniboia",
    lr=0.001,
    exp_decay=0.99,
    warmup_steps=1,
    wandb=None,
    task="boia",
    boia_model="ce",
    model="miniboialtn",
    c_sup=1,
    which_c=[-1],
    joint=False,
    boia_ood_knowledge=True,
)

# get dataset
dataset = get_dataset(args.dataset, args)
# get model
model = get_model(modelname=args.model, encoder=dataset.get_backbone()[0], args=args)

# set cpu for the moment
model.device = "cuda:2"

model.to(model.device)
if hasattr(model, "encoder"):
    model.encoder.to(model.device)
if hasattr(model, "net"):
    model.net.to(model.device)

model

Define the seeds of the models

In [ ]:
seeds = [123, 456, 789, 1011, 1213, 1415, 1617, 1819, 2021, 2223]
model_path = f"../best_model_{args.dataset}_{args.model}"

Loop through the dataset and retrive concepts and labels

In [ ]:
def get_concepts_and_labels_boia(out_labels, out_concepts):
    batch_size = out_labels.size(0)

    predicted_labels, predicted_concepts = [], []

    for idx_batch in range(batch_size):
        prob_labels = torch.split(out_labels[idx_batch], 2)
        prob_concepts = torch.split(out_concepts[idx_batch], 2)

        tmp_lab, tmp_conc = [], []

        for l_lab in prob_labels:
            tmp_lab.append(torch.argmax(l_lab, dim=0))
        for l_conc in prob_concepts:
            tmp_conc.append(torch.argmax(l_conc, dim=0))

        predicted_labels.append(torch.tensor([tmp_lab]))
        predicted_concepts.append(torch.tensor([tmp_conc]))

    predicted_labels = torch.concatenate(predicted_labels, dim=0)
    predicted_concepts = torch.concatenate(predicted_concepts, dim=0)

    return predicted_labels, predicted_concepts

In [ ]:
def get_concepts_and_labels_mnist(
    out_labels, out_concepts, true_concepts, is_ood=False
):

    # filtering out the extended support
    if not is_ood:
        for i in range(19):
            if i in [6, 10, 12]:
                continue
            out_labels[:, i] = 0

    predicted_labels = torch.argmax(out_labels, dim=-1)
    predicted_concepts = torch.argmax(out_concepts, dim=-1)

    predicted_concepts = predicted_concepts.view(predicted_concepts.numel())
    refactored_true_concepts = true_concepts.view(true_concepts.numel())

    return predicted_labels, predicted_concepts, refactored_true_concepts

In [ ]:
def get_concepts_and_labels_kand(out_labels, out_concepts, true_concepts):

    # take the prediction
    predicted_labels = torch.argmax(out_labels, dim=1)

    # stack colors and shapes on top of each other
    refactored_true_concepts = torch.split(true_concepts, 1, dim=1)
    refactored_true_concepts = torch.concatenate(
        refactored_true_concepts, dim=0
    ).squeeze(1)

    # take the prediction
    predicted_concepts_list = torch.split(out_concepts, 3, dim=2)
    predicted_concepts = []
    # take the argmax
    for pc in predicted_concepts_list:
        predicted_concepts.append(torch.argmax(pc, dim=2))
    predicted_concepts = torch.stack(predicted_concepts, dim=2)

    # make them the same dimension as the groundtruth
    predicted_concepts = torch.split(predicted_concepts, 1, dim=1)
    predicted_concepts = torch.concatenate(predicted_concepts, dim=0).squeeze(1)

    return predicted_labels, torch.squeeze(predicted_concepts), refactored_true_concepts

In [ ]:
def retrive_concepts_and_labels(model, dataset, dataset_name, is_ood=False):

    true_labels, predicted_labels, true_concepts, predicted_concepts = [], [], [], []

    nll_loss = 0.0
    criterion = torch.nn.CrossEntropyLoss(reduction="sum")

    for i, data in enumerate(tqdm(dataset)):
        images, labels, concepts = data
        images, labels, concepts = (
            images.to(model.device),
            labels.to(model.device),
            concepts.to(model.device),
        )

        # filtering out the middle rules supervision
        if dataset_name in ["kandinsky", "minikandinsky", "clipkandinsky"]:
            labels = labels[:, -1]

        out_dict = model(images)

        out_label, out_concept = None, None

        if dataset_name in ["boia", "miniboia", "clipboia", "clipminiboia"]:
            class_predictions = torch.split(out_dict["YS"], 2, dim=1)
            assert len(class_predictions) == 4

            loss = 0
            for i, _pred in enumerate(class_predictions):
                loss += criterion(_pred.float().cpu(), labels[:, i].long().cpu())
            loss /= len(class_predictions)
        else:
            loss = criterion(out_dict["YS"].float().cpu(), labels.long().cpu())

        nll_loss += loss.item()

        if dataset_name in ["boia", "miniboia", "clipboia", "clipminiboia"]:
            out_label, out_concept = get_concepts_and_labels_boia(
                out_dict["YS"], out_dict["pCS"]
            )
        elif dataset_name in ["shortmnist", "clipshortmnist"]:
            out_label, out_concept, concepts = get_concepts_and_labels_mnist(
                out_dict["YS"], out_dict["pCS"], concepts, is_ood
            )
        elif dataset_name in ["kandinsky", "minikandinsky", "clipkandinsky"]:
            out_label, out_concept, concepts = get_concepts_and_labels_kand(
                out_dict["YS"], out_dict["pCS"], concepts
            )

        true_labels.append(labels.cpu().numpy())
        true_concepts.append(concepts.cpu().numpy())

        predicted_labels.append(out_label.detach().cpu().numpy())
        predicted_concepts.append(out_concept.cpu().numpy())

    # concatenate
    true_labels = np.concatenate(true_labels, axis=0)
    predicted_labels = np.concatenate(predicted_labels, axis=0)
    true_concepts = np.concatenate(true_concepts, axis=0)
    predicted_concepts = np.concatenate(predicted_concepts, axis=0)

    avg_nll = nll_loss / len(dataset.dataset)

    assert true_labels.shape == predicted_labels.shape
    assert true_concepts.shape == predicted_concepts.shape

    return true_labels, predicted_labels, true_concepts, predicted_concepts, avg_nll

In [ ]:
def evaluate(
    model, test_set, dataset_name, model_name, ood_set=None, ood_set_2=None
):  # TODO: define attributes

    # List of metics
    in_metrics_list = []
    ood_metrics_list = []
    ood_metrics_2_list = []

    n_files = 0

    # Loop through seeds
    for seed in seeds:
        print("Doing", seed, "...")

        to_add = ""  # "_joint" # ""
        print("TO ADD", to_add)

        current_model_path = f"{model_path}_{seed}{to_add}.pth"

        if not os.path.exists(current_model_path):
            print(f"{current_model_path} is missing...")
            continue
        else:
            print(f"Loading {current_model_path}...")

        n_files += 1

        try:
            # retrieve the status dict
            model_state_dict = torch.load(current_model_path)
            # Load the model status dict
            model.load_state_dict(model_state_dict)
        except Exception as e:
            print(e)
            continue

        if dataset_name == "shortmnist":
            model = model.float()

        model.eval()

        ind_data = retrive_concepts_and_labels(model, test_set, dataset_name)

        if ood_set is not None:
            out_data = retrive_concepts_and_labels(
                model, ood_set, dataset_name, is_ood=True
            )

        if ood_set_2 is not None:
            out_data_2 = retrive_concepts_and_labels(
                model, ood_set_2, dataset_name, is_ood=True
            )

        in_metrics = compute_metrics(*ind_data, dataset_name, model_name, seed)
        in_metrics_list.append(in_metrics)

        if ood_set is not None:
            ood_metrics = compute_metrics(*out_data, dataset_name, model_name, seed)
            ood_metrics_list.append(ood_metrics)

        if ood_set_2 is not None:
            ood_metrics_2 = compute_metrics(*out_data_2, dataset_name, model_name, seed)
            ood_metrics_2_list.append(ood_metrics_2)

        torch.cuda.empty_cache()

    assert n_files > 1, "At least 2 files to compare"

    # Compute standard deviation for each metric
    for key in vars(in_metrics_list[0]):  # the key are always the same
        # skip hidden elements
        if not key.startswith("_"):
            # retrieve the list of values
            in_metric_values = [getattr(metrics, key) for metrics in in_metrics_list]
            ood_metric_values = [getattr(metrics, key) for metrics in ood_metrics_list]
            ood_metric_2_values = [
                getattr(metrics, key) for metrics in ood_metrics_2_list
            ]

            # convert lists to NumPy arrays
            in_metric_values_arr = np.array(in_metric_values)
            ood_metric_values_arr = np.array(ood_metric_values)
            ood_metric_values_2_arr = np.array(ood_metric_2_values)

            # Compute the standard deviation
            in_metric_std_dev = np.std(in_metric_values_arr)
            ood_metric_std_dev = np.std(ood_metric_values_arr)
            ood_metric_2_std_dev = np.std(ood_metric_values_2_arr)

            # Compute the mean
            in_metric_std_mean = np.mean(in_metric_values_arr)
            ood_metric_std_mean = np.mean(ood_metric_values_arr)
            ood_metric_2_std_mean = np.mean(ood_metric_values_2_arr)

            print(
                "\n{} (In): ${:.2f} \pm {:.2f}$".format(
                    key.replace("_", " ").title(),
                    round(in_metric_std_mean, 2),
                    round(in_metric_std_dev, 2),
                )
            )

            if ood_set is not None:
                print(
                    "{} (OOD): ${:.2f} \pm {:.2f}$".format(
                        key.replace("_", " ").title(),
                        round(ood_metric_std_mean, 2),
                        round(ood_metric_std_dev, 2),
                    )
                )

            if ood_set_2 is not None:
                print(
                    "{} (OOD 2): ${:.2f} \pm {:.2f}$".format(
                        key.replace("_", " ").title(),
                        round(ood_metric_2_std_mean, 2),
                        round(ood_metric_2_std_dev, 2),
                    )
                )

Run all the things

In [ ]:
# Get loaders
train_loader, val_loader, test_loader = dataset.get_data_loaders()
# Get ood set if it exists
ood_loader = getattr(dataset, "ood_loader_ambulance", None)
# ood_ambulance = getattr(dataset, "ood_loader_2", None) # getattr(dataset, "ood_loader_ambulance", None)

# Evaluate
evaluate(
    model,
    test_loader,
    args.dataset,
    model_name=args.model,
    ood_set=ood_loader,
    ood_set_2=None,
)